# Credit risk prediction, explainability and bias detection with Amazon SageMaker

![Credit risk explainability use case](credit_risk_prediction.png)

1. [Overview](#Overview)
1. [Prerequisites and Data](#Prerequisites-and-Data)
    1. [Initialize SageMaker](#Initialize-SageMaker)
    1. [Download data](#Download-data)
    1. [Loading the data: German credit (Update) Dataset](#Loading-the-data:-German-credit-Dataset) 
    1. [Data inspection](#Data-inspection) 
    1. [Data preprocessing Model and upload to S3](#Preprocess-and-Upload-Training-Data) 
1. [Train XGBoost Model](#Train-XGBoost-Model)
    1. [Train Model](#Train-Model)
1. [Create SageMaker Model with Inference Pipeline](#Create-SageMaker-Model)
1. [Amazon SageMaker Clarify](#Amazon-SageMaker-Clarify)
    1. [Explaining Predictions](#Explaining-Predictions)
        1. [Viewing the Explainability Report](#Viewing-the-Explainability-Report)
        2. [Explaining individual bad credit prediction example](#Explaining-individual-prediction)
    2. [Understanding Bias](#Bias-Detection)
        1. [Pre-training bias metrics](#pre-training)
        2. [Post-training bias metrics](#post-training)
1. [Clean Up](#Clean-Up)
1. [Additional Resources](#Additional-Resources)

## 1. Overview
Amazon SageMaker helps data scientists and developers to prepare, build, train, and deploy high-quality machine learning (ML) models quickly by bringing together a broad set of capabilities purpose-built for ML.

[Amazon SageMaker Clarify](https://aws.amazon.com/sagemaker/clarify/) helps improve your machine learning models by detecting potential bias and helping explain how these models make predictions. The fairness and explainability functionality provided by SageMaker Clarify takes a step towards enabling AWS customers to build trustworthy and understandable machine learning models. 

Amazon SageMaker provides pre-made images for machine and deep learning frameworks for supported frameworks such as Scikit-Learn, XGBoost, TensorFlow, PyTorch, MXNet, or Chainer. These are preloaded with the corresponding framework and some additional Python packages, such as Pandas and NumPy, so you can write your own code for model training. See [here](https://docs.aws.amazon.com/sagemaker/latest/dg/algorithms-choose.html#supported-frameworks-benefits) for more information.


[Amazon SageMaker Studio](https://aws.amazon.com/sagemaker/studio/) provides a single, web-based visual interface where you can perform all ML development activities including notebooks, experiment management, automatic model creation, debugging, and model and data drift detection.

In this SageMaker Studio notebook, we highlight how you can use SageMaker to train models, create a deployable SageMaker model, and provide bias detection and explainability to analyze data and understand prediction outcomes from the model.
This sample notebook walks you through:  

1. Download and explore credit risk dataset - [South German Credit (UPDATE) Data Set](https://archive.ics.uci.edu/ml/datasets/South+German+Credit+%28UPDATE%29)
2. Preprocessing data with sklearn on the dataset
3. Training GBM model with XGBoost on the dataset
4. Build an inference pipeline model (sklearn model and XGBoost model together) to preprocess input data and produce a prediction outcome per instance
5. Hosting and scoring the single model (Optional)
6. SageMaker Clarify job to provide Kernel SHAP values for the SageMaker model on training and test datasets.
7. SageMaker Clarify job to provide bias metrics including pre-training bias metrics on data, 

![Credit risk explainability model inference](clarify_inf_pipeline_arch.jpg)

## 2. Prerequisites and Data exploration and Feature engineering
### Initialize SageMaker

In [1]:
# cell 01
from io import StringIO
import os
import time
import sys
import IPython
from time import gmtime, strftime

import boto3
import numpy as np
import pandas as pd
import urllib

import sagemaker
from sagemaker.s3 import S3Uploader
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost import XGBoost
from sagemaker.s3 import S3Downloader
from sagemaker.s3 import S3Uploader
from sagemaker import Session
from sagemaker import get_execution_role
from sagemaker.xgboost import XGBoostModel
from sagemaker.sklearn import SKLearnModel
from sagemaker.pipeline import PipelineModel


session = Session()
bucket = session.default_bucket()
prefix = "sagemaker/pisa-2022-prediction-model"
region = session.boto_region_name

# Define IAM role
role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## 5. Amazon SageMaker Clarify

Pre-requisities :

1. [SageMaker Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html) that can be deployed to a endpoint

2. Input dataset
                  
3. SHAP Baseline

Now that you have your model set up. Let's say hello to SageMaker Clarify!

In [2]:
# cell 25
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, instance_count=1, instance_type="ml.c4.xlarge", sagemaker_session=session
)

### Explaining Predictions
There are expanding business needs and legislative regulations that require explanations of _why_ a model made the decision it did. SageMaker Clarify uses [SHAP library](https://github.com/slundberg/shap) to explain the contribution that each input feature makes to the final decision. SageMaker Clarify uses a scalable and efficient implementation of [Kernel SHAP](https://github.com/slundberg/shap#model-agnostic-example-with-kernelexplainer-explains-any-function) with an option to use spark based parallelization with multiple processing instances. Note that Kernel SHAP and hence SageMaker Clarify has a model-agnostic feature attribution approach. Any ML model that is represented as a [SageMaker model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html) can be used with Clarify for explainability. 

Here is more information about explainability with Clarify and SHAP:

    https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-explainability.html
    
    https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-shapley-values.html
    
    https://papers.nips.cc/paper/2017/file/8a20a8621978632d76c43dfd28b67767-Paper.pdf

### Create a baseline for SHAP

As a contrastive explainability technique, SHAP values are calculated by evaluating the model on synthetic data generated against a baseline sample. The explanations of the same case can be different depending on the choices of this baseline sample. 

We are interested in explaining bad credit predictions. Hence, we would like the baseline choice to have E(x) closer to 1(belonging to the good credit class). 

We use the [mode](https://en.wikipedia.org/wiki/Mode_(statistics)) statistic to create the baseline. The mode is a good choice for categorical variables. We observe that the model prediction for the baseline has a high probability for the good credit class and hence it satisfies our requirement for the baseline. 

For more information on selecting informative vs non-informative baselines, see [SHAP Baselines for Explainability ](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html)

In [3]:
# cell 26
# load the raw training data in a data frame
raw_train_df = pd.read_csv("train.csv", header=0, names=None, sep=",")

KeyError: "['MATH_Proficient'] not found in axis"

In [5]:
# drop the target column
# Drop the first column (MATH_Proficient) when CSV has no headers
baseline = raw_train_df.drop(raw_train_df.columns[0], axis=1).mode().iloc[0].values.astype("int").tolist()
print(baseline)

[49900035, 800001, 1, 4, 1, 0, 0, 0, 0, 0, 1, 3, 5, 5, 5, 1, 1, 1, 3, 8, 10, 8, 2, -2, 2, 0, 0, 9, 7, 6, 4, 1, 0, 1, 10, 0, 0, 0, 16, 17, 25, 2, 1, 4, 0, 2, 2, 30, 1, 2, 10, 1, 4, 1, 1, 4, 1, 8, 2, 2, 4, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 1, 2, 1, 6, 6, 2, 5, 3, 2, 3, 3, 1, 1, 2, 3, 1, 4, 1, 4, 2, 2, 1, 2, 3, 1, 3, 10, 20, 0, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, -1, 0, 0, 0, 0, 2, 2, -1, -1, 0, 0, 0, 0, -1, 1, -2, -2, 0, 0, 0, 0, 0, 0, 0, 0, -2, -2, -1, 0, 0, 0, -2, 0, 2, 0, 1, 0, -2, 0, 1, 4, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 3, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 1, 1, 45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 4, 4, 1, 1, 4, 4, 3, 3, 1, 2, 1, 1, 100, 0, 1, 2, 1, 1, 100, 23, 1, 0, 0, 0, 0, 3, 10, 100, 10, 100, 0, 5, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 0, 3, 0, 0, 1, 2, 2, 4, 5, 2, 1, 4, 5, 3, 3, 5, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 11, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, -1, -1,

In [28]:
test_data = raw_train_df.drop(raw_train_df.columns[0], axis=1).sample(frac=0.1)
test_uo_filename = "test_unordered.csv"
test_data.to_csv(test_uo_filename, index=False, header=True, columns=None, sep=",")
# Save train dataset with MATH_Proficient as the first column
cols = list(test_data.columns)
cols = [cols[2]] + cols[:2] + cols[3:]  # Move 3rd column to the front
test_filename = "test.csv"
test_data[cols].to_csv(test_filename, index=False, header=False)

In [33]:
test_data[cols].head
test_data[cols].shape

(41430, 568)

In [29]:
test_raw = S3Uploader.upload(test_filename, "s3://{}/{}/data/test".format(bucket, prefix))
print(test_raw)

s3://sagemaker-us-west-2-986030204467/sagemaker/pisa-2022-prediction-model/data/test/test.csv


In [30]:
raw_train_df.head

<bound method NDFrame.head of         0  21400051  21407678  1.0  3.0  3.0.1  0.1  0.2  1  0.3  ...  0.238  \
0       0  18800072  18804079  1.0  5.0    2.0    0    1  0    0  ...      0   
1       1  45800120  45809473  1.0  5.0    1.0    0    0  1    0  ...      0   
2       1  12400760  12413119  1.0  2.0    1.0    0    1  0    0  ...      0   
3       0  32000023  32002783  NaN  NaN    NaN    0    0  0    0  ...      0   
4       0  60800039  60806443  1.0  6.0    1.0    0    1  0    0  ...      0   
...    ..       ...       ...  ...  ...    ...  ...  ... ..  ...  ...    ...   
414293  0  40000040  40000237  1.0  6.0    1.0    0    0  1    0  ...      0   
414294  1  25000103  25003470  1.0  4.0    1.0    0    0  0    1  ...      0   
414295  0  11600057  11605138  1.0  4.0    4.0    0    0  1    0  ...      0   
414296  1  60800141  60800538  NaN  NaN    NaN    0    0  0    0  ...      0   
414297  0    800145    804599  NaN  NaN    NaN    0    0  0    0  ...      0   

        0

# Find best model from Hyper Parameter Tuning Job

In [8]:
sm_client = boto3.client('sagemaker')

# Replace with your tuning job name
tuning_job_name = 'xgboost-250217-0415'

# Get the best model from the tuning job
best_training_job = sm_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)['BestTrainingJob']
model_name = best_training_job['TrainingJobName']

print(f"Best model: {model_name}")

Best model: xgboost-250217-0415-003-45af3f90


In [43]:
from sagemaker.tuner import HyperparameterTuner

tuner = HyperparameterTuner.attach('xgboost-250217-0415') 
#  Deploy the best trained or user specified model to an Amazon SageMaker endpoint
tuner_predictor = tuner.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')


2025-02-17 04:29:54 Starting - Preparing the instances for training
2025-02-17 04:29:54 Downloading - Downloading the training image
2025-02-17 04:29:54 Training - Training image download completed. Training in progress.
2025-02-17 04:29:54 Uploading - Uploading generated training model
2025-02-17 04:29:54 Completed - Resource released due to keep alive period expiry

INFO:sagemaker:Creating model with name: xgboost-2025-02-17-06-15-59-294


INFO:sagemaker:Creating endpoint-config with name xgboost-250217-0415-003-45af3f90
INFO:sagemaker:Creating endpoint with name xgboost-250217-0415-003-45af3f90


-------!

In [45]:
# cell 27
# check baseline prediction E[f(x)]
predictor = sagemaker.predictor.Predictor(
    tuner_predictor.endpoint_name,
    session,
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer(),
)

pred_baseline = predictor.predict(baseline)
print(pred_baseline)

[['0.6958543062210083']]


### Setup configurations for Clarify

Next, setup some more configurations to start the explainability analysis by Clarify. We need to set up the following:
  1. **SHAPConfig**: to create the baseline. In this example, the mean_abs is the mean of absolute SHAP values for all instances, specified as the baseline 
  1. **DataConfig**: to provide some basic information about data I/O to SageMaker Clarify. We specify where to find the input dataset, where to store the output, the header names, and the dataset type.
  1. **ModelConfig**: to specify information about the trained model here we re-use the model name created earlier. 
  
  Note: To avoid additional traffic to your production models, SageMaker Clarify sets up and tears down a ephemeral endpoint while processing. ModelConfig specifies your preferred instance type and instance count used to run your model on during Clarify's processing.
  
To know more about what these configurations mean for Clarify, check out the documentation here: https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html

2 × 560 feature = 1120
Add 2000 2000 as a baseline → 2000 + 1120 = 3120
2000+1120=3120

In [35]:
# cell 28
shap_config = clarify.SHAPConfig(
    baseline=[baseline],
    num_samples=3120,  # num_samples are permutations from your features, so should be large enough as compared to number of input features, for example, 2k + 2* num_features
    agg_method="mean_abs",
    use_logit=True,
)  # we want the shap values to have log-odds units so that the equation 'shap values + expected probability =  predicted probability' for each instance record )

In [36]:
test_columns = [
    "CNTSCHID", "CNTSTUID", "SISCO", "ST347Q01JA", "ST347Q02JA", "ST349Q01JA_0", "ST349Q01JA_1", "ST349Q01JA_2", "ST349Q01JA_3", "ST349Q01JA_4", "ST350Q01JA", "ST356Q01JA", "ST322Q01JA", "ST322Q02JA", "ST322Q03JA", "ST322Q04JA", "ST322Q06JA", "ST322Q07JA", "DURECEC", "EFFORT1", "EFFORT2", "ST259Q01JA",
    "WB164Q01HA", "HOMEPOS", "ST004D01T", "GRADE", "REPEAT", "EXPECEDU", "ICTAVSCH", "ICTAVHOM", "ICTDISTR", "IMMIG", "TARDYSD", "ST226Q01JA", "ST016Q01NA", "MISSSC", "Option_UH", "OECD", "PAREDINT", "BMMJ1", "BFMJ2", "WB163Q06HA",
    "WB163Q07HA", "ST230Q01JA", "SKIPPING", "IC180Q01JA", "IC180Q08JA", "ST059Q02JA", "ST296Q04JA", "WB176Q01HA", "STUDYHMW", "IC184Q01JA", "IC184Q02JA", "IC184Q03JA", "IC184Q04JA", "ST059Q01TA", "ST296Q01JA", "ST272Q01JA", "ST268Q01JA", "ST268Q04JA", "ST268Q07JA", "ST293Q04JA",
    "ST297Q01JA", "ST297Q03JA", "ST297Q05JA", "ST297Q06JA", "ST297Q07JA", "ST297Q09JA", "WB165Q01HA", "WB166Q01HA", "WB166Q02HA", "WB166Q03HA", "WB166Q04HA", "ST258Q01JA", "ST294Q01JA", "ST295Q01JA", "WB150Q01HA", "WB156Q01HA", "WB158Q01HA", "WB160Q01HA", "WB161Q01HA", "WB171Q01HA",
    "WB171Q02HA", "WB171Q03HA", "WB171Q04HA", "WB172Q01HA", "WB173Q01HA", "WB173Q02HA", "WB173Q03HA", "WB173Q04HA", "WB177Q01HA", "WB177Q02HA", "WB177Q03HA", "WB177Q04HA", "WB032Q01NA", "WB032Q02NA", "WB031Q01NA", "EXERPRAC", "STUBMI", "RELATST", "BELONG", "BULLIED",
    "FEELSAFE", "SCHRISK", "PERSEVAGR", "CURIOAGR", "COOPAGR", "EMPATAGR", "ASSERAGR", "STRESAGR", "EMOCOAGR", "GROSAGR", "INFOSEEK", "FAMSUP", "DISCLIM", "TEACHSUP", "COGACRCO", "COGACMCO", "EXPOFA", "EXPO21ST", "MATHEFF", "MATHEF21",
    "FAMCON", "ANXMAT", "MATHPERS", "CREATEFF", "CREATSCH", "CREATFAM", "CREATAS", "CREATOOS", "CREATOP", "OPENART", "IMAGINE", "SCHSUST", "LEARRES", "PROBSELF", "FAMSUPSL", "FEELLAH", "SDLEFF", "ICTRES", "ESCS", "FLSCHOOL",
    "FLMULTSB", "FLFAMILY", "ACCESSFP", "FLCONFIN", "FLCONICT", "ACCESSFA", "ATTCONFM", "FRINFLFM", "ICTSCH", "ICTHOME", "ICTQUAL", "ICTSUBJ", "ICTENQ", "ICTFEED", "ICTOUT", "ICTWKDY", "ICTWKEND", "ICTREG", "ICTINFO", "ICTEFFIC",
    "BODYIMA", "SOCONPA", "LIFESAT", "PSYCHSYM", "SOCCON", "EXPWB", "CURSUPP", "PQMIMP", "PQMCAR", "PARINVOL", "PQSCHOOL", "PASCHPOL", "ATTIMMP", "CREATHME", "CREATACT", "CREATOPN", "CREATOR", "WORKPAY", "WORKHOME", "SC001Q01TA",
    "SC211Q01JA", "SC211Q02JA", "SC211Q03JA", "SC211Q04JA", "SC211Q05JA", "SC211Q06JA", "SC209Q04JA", "SC209Q05JA", "SC209Q06JA", "SC037Q11JA", "SC183Q02JA", "SC183Q03JA", "SC183Q04JA", "SC175Q01JA", "SC177Q01JA_1", "SC177Q01JA_2", "SC177Q01JA_3", "SC177Q02JA_1", "SC177Q02JA_2", "SC177Q02JA_3",
    "SC177Q03JA_1", "SC177Q03JA_2", "SC177Q03JA_3", "SC188Q01JA", "SC188Q02JA", "SC188Q03JA", "SC188Q04JA", "SC188Q05JA", "SC188Q06JA", "SC188Q07JA", "SC188Q08JA", "SC188Q09JA", "SC188Q10JA", "SC188Q11JA", "SC198Q01JA", "SC198Q02JA", "SC198Q03JA", "SC178Q01JA", "SC178Q02JA", "SC180Q01JA",
    "SC189Q02WA", "SC189Q03WA", "SC189Q04WA", "SMRATIO", "MCLSIZE", "MACTIV", "MATHEXC_0", "MATHEXC_1", "MATHEXC_2", "MATHEXC_3", "ABGMATH", "SC064Q05WA", "SC064Q06WA", "SC064Q01TA", "SC064Q02TA", "SC064Q04NA", "SC064Q03TA", "SC064Q07WA", "SC213Q01JA", "SC213Q02JA",
    "SC037Q01TA", "SC037Q02TA", "SC037Q03TA", "SC037Q04TA", "SC037Q05NA", "SC037Q06NA", "SC037Q07TA", "SC037Q09TA", "SC200Q01JA", "SC200Q02JA", "SC200Q03JA", "SC200Q04JA", "SC224Q01JA", "RATCMP1", "RATCMP2", "RATTAB", "SCHSEL", "SCHLTYPE_1", "SCHLTYPE_2", "SCHLTYPE_3",
    "SC034Q01NA", "SC034Q02NA", "SC034Q03TA", "SC034Q04TA", "SC195Q01JA", "SC195Q02JA", "SC195Q03JA", "SC195Q04JA", "SC042Q01TA", "SC042Q02TA", "SC214Q01JA", "SC214Q02JA", "SC214Q03JA", "SC215Q01JA", "SC215Q02JA", "SC215Q03JA", "SC215Q04JA", "SC215Q05JA", "SC215Q06JA", "SC215Q07JA",
    "SC215Q08JA", "SC216Q06JA", "SC216Q07JA", "SC216Q08JA", "SC216Q09JA", "SC217Q01JA", "SC217Q02JA", "SC217Q03JA", "SC217Q04JA", "SC217Q05JA", "SC217Q06JA", "SC217Q07JA", "SC217Q08JA", "SC217Q10JA", "SC218Q01JA", "SC219Q01JA", "SC220Q01JA", "SC221Q01JA", "SC221Q02JA", "SC221Q03JA",
    "SC221Q04JA", "SCSUPRTED", "SCSUPRT", "SC212Q01JA", "SC212Q02JA", "SC212Q03JA", "SC037Q08TA", "SC032Q01TA", "SC032Q02TA", "SC032Q03TA", "SC032Q04TA", "SCHAUTO", "TCHPART", "EDULEAD", "INSTLEAD", "ENCOURPG", "DIGDVPOL", "TEAFDBK", "MTTRAIN", "DMCVIEWS",
    "NEGSCLIM", "STAFFSHORT", "EDUSHORT", "STUBEHA", "TEACHBEHA", "STDTEST", "TDTEST", "ALLACTIV", "BCREATSC", "CREENVSC", "ACTCRESC", "OPENCUL", "PROBSCRI", "SCPREPBP", "SCPREPAP", "DIGPREP", "LANGN_105", "LANGN_108", "LANGN_112", "LANGN_113",
    "LANGN_118", "LANGN_121", "LANGN_130", "LANGN_133", "LANGN_137", "LANGN_140", "LANGN_147", "LANGN_148", "LANGN_150", "LANGN_154", "LANGN_156", "LANGN_160", "LANGN_170", "LANGN_195", "LANGN_200", "LANGN_202", "LANGN_204", "LANGN_232", "LANGN_237", "LANGN_244",
    "LANGN_246", "LANGN_254", "LANGN_258", "LANGN_263", "LANGN_264", "LANGN_266", "LANGN_272", "LANGN_273", "LANGN_275", "LANGN_286", "LANGN_301", "LANGN_313", "LANGN_316", "LANGN_317", "LANGN_322", "LANGN_325", "LANGN_327", "LANGN_329", "LANGN_338", "LANGN_340",
    "LANGN_344", "LANGN_351", "LANGN_358", "LANGN_363", "LANGN_369", "LANGN_371", "LANGN_375", "LANGN_379", "LANGN_381", "LANGN_382", "LANGN_383", "LANGN_404", "LANGN_409", "LANGN_415", "LANGN_420", "LANGN_422", "LANGN_428", "LANGN_434", "LANGN_442", "LANGN_449",
    "LANGN_451", "LANGN_463", "LANGN_465", "LANGN_467", "LANGN_471", "LANGN_472", "LANGN_474", "LANGN_492", "LANGN_493", "LANGN_494", "LANGN_495", "LANGN_496", "LANGN_500", "LANGN_503", "LANGN_514", "LANGN_517", "LANGN_520", "LANGN_523", "LANGN_527", "LANGN_529",
    "LANGN_531", "LANGN_540", "LANGN_547", "LANGN_555", "LANGN_561", "LANGN_562", "LANGN_563", "LANGN_565", "LANGN_566", "LANGN_567", "LANGN_600", "LANGN_601", "LANGN_602", "LANGN_605", "LANGN_606", "LANGN_607", "LANGN_608", "LANGN_611", "LANGN_614", "LANGN_615",
    "LANGN_616", "LANGN_618", "LANGN_619", "LANGN_621", "LANGN_622", "LANGN_623", "LANGN_624", "LANGN_625", "LANGN_626", "LANGN_627", "LANGN_628", "LANGN_630", "LANGN_631", "LANGN_634", "LANGN_635", "LANGN_639", "LANGN_640", "LANGN_641", "LANGN_642", "LANGN_648",
    "LANGN_650", "LANGN_661", "LANGN_662", "LANGN_663", "LANGN_665", "LANGN_666", "LANGN_667", "LANGN_668", "LANGN_669", "LANGN_670", "LANGN_673", "LANGN_674", "LANGN_675", "LANGN_676", "LANGN_677", "LANGN_678", "LANGN_800", "LANGN_801", "LANGN_802", "LANGN_804",
    "LANGN_805", "LANGN_806", "LANGN_807", "LANGN_808", "LANGN_809", "LANGN_810", "LANGN_811", "LANGN_812", "LANGN_813", "LANGN_814", "LANGN_815", "LANGN_816", "LANGN_817", "LANGN_818", "LANGN_819", "LANGN_821", "LANGN_823", "LANGN_824", "LANGN_825", "LANGN_826",
    "LANGN_827", "LANGN_828", "LANGN_829", "LANGN_831", "LANGN_832", "LANGN_833", "LANGN_836", "LANGN_837", "LANGN_838", "LANGN_839", "LANGN_840", "LANGN_841", "LANGN_842", "LANGN_843", "LANGN_844", "LANGN_845", "LANGN_846", "LANGN_849", "LANGN_850", "LANGN_851",
    "LANGN_852", "LANGN_854", "LANGN_855", "LANGN_857", "LANGN_859", "LANGN_860", "LANGN_861", "LANGN_865", "LANGN_866", "LANGN_868", "LANGN_870", "LANGN_872", "LANGN_873", "LANGN_877", "LANGN_879", "LANGN_881", "LANGN_885", "LANGN_890", "LANGN_892", "LANGN_895",
    "LANGN_896", "LANGN_897", "LANGN_898", "LANGN_899", "LANGN_900", "LANGN_901", "LANGN_902", "LANGN_903", "LANGN_904", "LANGN_905", "LANGN_906", "LANGN_907", "LANGN_908", "LANGN_909", "LANGN_910", "LANGN_911", "LANGN_912", "LANGN_913", "LANGN_914", "LANGN_916",
    "LANGN_917", "LANGN_918", "LANGN_919", "LANGN_920", "LANGN_921", "LANGN_922"
]

In [37]:
# cell 29
explainability_report_prefix = "{}/clarify-explainability".format(prefix)
explainability_output_path = "s3://{}/{}".format(bucket, explainability_report_prefix)

explainability_data_config = clarify.DataConfig(
    s3_data_input_path=test_raw,
    s3_output_path=explainability_output_path,
    # label='credit_risk', # target column is not present in the test dataset
    headers=test_columns,
    dataset_type="text/csv",
)

In [34]:
def csv_string_to_python_array(csv_string):
    variables = [var.strip() for var in csv_string.split(',')]
    lines = []
    for i in range(0, len(variables), 20):
        line = ", ".join(f'"{var}"' for var in variables[i:i+20])
        lines.append(line)

    array_text = "[\n    " + ",\n    ".join(lines) + "\n]"

    print("Python array:")
    print(array_text)

# Example usage
csv_input = "CNTSCHID, CNTSTUID,SISCO,ST347Q01JA,ST347Q02JA,ST349Q01JA_0,ST349Q01JA_1,ST349Q01JA_2,ST349Q01JA_3,ST349Q01JA_4,ST350Q01JA,ST356Q01JA,ST322Q01JA,ST322Q02JA,ST322Q03JA,ST322Q04JA,ST322Q06JA,ST322Q07JA,DURECEC,EFFORT1,EFFORT2,ST259Q01JA,WB164Q01HA,HOMEPOS,ST004D01T,GRADE,REPEAT,EXPECEDU,ICTAVSCH,ICTAVHOM,ICTDISTR,IMMIG,TARDYSD,ST226Q01JA,ST016Q01NA,MISSSC,Option_UH,OECD,PAREDINT,BMMJ1,BFMJ2,WB163Q06HA,WB163Q07HA,ST230Q01JA,SKIPPING,IC180Q01JA,IC180Q08JA,ST059Q02JA,ST296Q04JA,WB176Q01HA,STUDYHMW,IC184Q01JA,IC184Q02JA,IC184Q03JA,IC184Q04JA,ST059Q01TA,ST296Q01JA,ST272Q01JA,ST268Q01JA,ST268Q04JA,ST268Q07JA,ST293Q04JA,ST297Q01JA,ST297Q03JA,ST297Q05JA,ST297Q06JA,ST297Q07JA,ST297Q09JA,WB165Q01HA,WB166Q01HA,WB166Q02HA,WB166Q03HA,WB166Q04HA,ST258Q01JA,ST294Q01JA,ST295Q01JA,WB150Q01HA,WB156Q01HA,WB158Q01HA,WB160Q01HA,WB161Q01HA,WB171Q01HA,WB171Q02HA,WB171Q03HA,WB171Q04HA,WB172Q01HA,WB173Q01HA,WB173Q02HA,WB173Q03HA,WB173Q04HA,WB177Q01HA,WB177Q02HA,WB177Q03HA,WB177Q04HA,WB032Q01NA,WB032Q02NA,WB031Q01NA,EXERPRAC,STUBMI,RELATST,BELONG,BULLIED,FEELSAFE,SCHRISK,PERSEVAGR,CURIOAGR,COOPAGR,EMPATAGR,ASSERAGR,STRESAGR,EMOCOAGR,GROSAGR,INFOSEEK,FAMSUP,DISCLIM,TEACHSUP,COGACRCO,COGACMCO,EXPOFA,EXPO21ST,MATHEFF,MATHEF21,FAMCON,ANXMAT,MATHPERS,CREATEFF,CREATSCH,CREATFAM,CREATAS,CREATOOS,CREATOP,OPENART,IMAGINE,SCHSUST,LEARRES,PROBSELF,FAMSUPSL,FEELLAH,SDLEFF,ICTRES,ESCS,FLSCHOOL,FLMULTSB,FLFAMILY,ACCESSFP,FLCONFIN,FLCONICT,ACCESSFA,ATTCONFM,FRINFLFM,ICTSCH,ICTHOME,ICTQUAL,ICTSUBJ,ICTENQ,ICTFEED,ICTOUT,ICTWKDY,ICTWKEND,ICTREG,ICTINFO,ICTEFFIC,BODYIMA,SOCONPA,LIFESAT,PSYCHSYM,SOCCON,EXPWB,CURSUPP,PQMIMP,PQMCAR,PARINVOL,PQSCHOOL,PASCHPOL,ATTIMMP,CREATHME,CREATACT,CREATOPN,CREATOR,WORKPAY,WORKHOME,SC001Q01TA,SC211Q01JA,SC211Q02JA,SC211Q03JA,SC211Q04JA,SC211Q05JA,SC211Q06JA,SC209Q04JA,SC209Q05JA,SC209Q06JA,SC037Q11JA,SC183Q02JA,SC183Q03JA,SC183Q04JA,SC175Q01JA,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,SC188Q01JA,SC188Q02JA,SC188Q03JA,SC188Q04JA,SC188Q05JA,SC188Q06JA,SC188Q07JA,SC188Q08JA,SC188Q09JA,SC188Q10JA,SC188Q11JA,SC198Q01JA,SC198Q02JA,SC198Q03JA,SC178Q01JA,SC178Q02JA,SC180Q01JA,SC189Q02WA,SC189Q03WA,SC189Q04WA,SMRATIO,MCLSIZE,MACTIV,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,ABGMATH,SC064Q05WA,SC064Q06WA,SC064Q01TA,SC064Q02TA,SC064Q04NA,SC064Q03TA,SC064Q07WA,SC213Q01JA,SC213Q02JA,SC037Q01TA,SC037Q02TA,SC037Q03TA,SC037Q04TA,SC037Q05NA,SC037Q06NA,SC037Q07TA,SC037Q09TA,SC200Q01JA,SC200Q02JA,SC200Q03JA,SC200Q04JA,SC224Q01JA,RATCMP1,RATCMP2,RATTAB,SCHSEL,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,SC034Q01NA,SC034Q02NA,SC034Q03TA,SC034Q04TA,SC195Q01JA,SC195Q02JA,SC195Q03JA,SC195Q04JA,SC042Q01TA,SC042Q02TA,SC214Q01JA,SC214Q02JA,SC214Q03JA,SC215Q01JA,SC215Q02JA,SC215Q03JA,SC215Q04JA,SC215Q05JA,SC215Q06JA,SC215Q07JA,SC215Q08JA,SC216Q06JA,SC216Q07JA,SC216Q08JA,SC216Q09JA,SC217Q01JA,SC217Q02JA,SC217Q03JA,SC217Q04JA,SC217Q05JA,SC217Q06JA,SC217Q07JA,SC217Q08JA,SC217Q10JA,SC218Q01JA,SC219Q01JA,SC220Q01JA,SC221Q01JA,SC221Q02JA,SC221Q03JA,SC221Q04JA,SCSUPRTED,SCSUPRT,SC212Q01JA,SC212Q02JA,SC212Q03JA,SC037Q08TA,SC032Q01TA,SC032Q02TA,SC032Q03TA,SC032Q04TA,SCHAUTO,TCHPART,EDULEAD,INSTLEAD,ENCOURPG,DIGDVPOL,TEAFDBK,MTTRAIN,DMCVIEWS,NEGSCLIM,STAFFSHORT,EDUSHORT,STUBEHA,TEACHBEHA,STDTEST,TDTEST,ALLACTIV,BCREATSC,CREENVSC,ACTCRESC,OPENCUL,PROBSCRI,SCPREPBP,SCPREPAP,DIGPREP,LANGN_105,LANGN_108,LANGN_112,LANGN_113,LANGN_118,LANGN_121,LANGN_130,LANGN_133,LANGN_137,LANGN_140,LANGN_147,LANGN_148,LANGN_150,LANGN_154,LANGN_156,LANGN_160,LANGN_170,LANGN_195,LANGN_200,LANGN_202,LANGN_204,LANGN_232,LANGN_237,LANGN_244,LANGN_246,LANGN_254,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_272,LANGN_273,LANGN_275,LANGN_286,LANGN_301,LANGN_313,LANGN_316,LANGN_317,LANGN_322,LANGN_325,LANGN_327,LANGN_329,LANGN_338,LANGN_340,LANGN_344,LANGN_351,LANGN_358,LANGN_363,LANGN_369,LANGN_371,LANGN_375,LANGN_379,LANGN_381,LANGN_382,LANGN_383,LANGN_404,LANGN_409,LANGN_415,LANGN_420,LANGN_422,LANGN_428,LANGN_434,LANGN_442,LANGN_449,LANGN_451,LANGN_463,LANGN_465,LANGN_467,LANGN_471,LANGN_472,LANGN_474,LANGN_492,LANGN_493,LANGN_494,LANGN_495,LANGN_496,LANGN_500,LANGN_503,LANGN_514,LANGN_517,LANGN_520,LANGN_523,LANGN_527,LANGN_529,LANGN_531,LANGN_540,LANGN_547,LANGN_555,LANGN_561,LANGN_562,LANGN_563,LANGN_565,LANGN_566,LANGN_567,LANGN_600,LANGN_601,LANGN_602,LANGN_605,LANGN_606,LANGN_607,LANGN_608,LANGN_611,LANGN_614,LANGN_615,LANGN_616,LANGN_618,LANGN_619,LANGN_621,LANGN_622,LANGN_623,LANGN_624,LANGN_625,LANGN_626,LANGN_627,LANGN_628,LANGN_630,LANGN_631,LANGN_634,LANGN_635,LANGN_639,LANGN_640,LANGN_641,LANGN_642,LANGN_648,LANGN_650,LANGN_661,LANGN_662,LANGN_663,LANGN_665,LANGN_666,LANGN_667,LANGN_668,LANGN_669,LANGN_670,LANGN_673,LANGN_674,LANGN_675,LANGN_676,LANGN_677,LANGN_678,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_817,LANGN_818,LANGN_819,LANGN_821,LANGN_823,LANGN_824,LANGN_825,LANGN_826,LANGN_827,LANGN_828,LANGN_829,LANGN_831,LANGN_832,LANGN_833,LANGN_836,LANGN_837,LANGN_838,LANGN_839,LANGN_840,LANGN_841,LANGN_842,LANGN_843,LANGN_844,LANGN_845,LANGN_846,LANGN_849,LANGN_850,LANGN_851,LANGN_852,LANGN_854,LANGN_855,LANGN_857,LANGN_859,LANGN_860,LANGN_861,LANGN_865,LANGN_866,LANGN_868,LANGN_870,LANGN_872,LANGN_873,LANGN_877,LANGN_879,LANGN_881,LANGN_885,LANGN_890,LANGN_892,LANGN_895,LANGN_896,LANGN_897,LANGN_898,LANGN_899,LANGN_900,LANGN_901,LANGN_902,LANGN_903,LANGN_904,LANGN_905,LANGN_906,LANGN_907,LANGN_908,LANGN_909,LANGN_910,LANGN_911,LANGN_912,LANGN_913,LANGN_914,LANGN_916,LANGN_917,LANGN_918,LANGN_919,LANGN_920,LANGN_921,LANGN_922"
#csv_string_to_python_array(csv_input)

In [38]:
# cell 30
model_config = clarify.ModelConfig(
    model_name=tuner_predictor.endpoint_name,  # specify the inference pipeline model name
    instance_type="ml.c5.xlarge",
    instance_count=1,
    accept_type="text/csv",
)

In [49]:
tuner_predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: xgboost-250217-0415-003-45af3f90
INFO:sagemaker:Deleting endpoint with name: xgboost-250217-0415-003-45af3f90


### Run SageMaker Clarify Explainability job

All the configurations are in place. Let's start the explainability job. This will spin up an ephemeral SageMaker endpoint and perform inference and calculate explanations on that endpoint. It does not use any existing production endpoint deployments.

#### NOTE: THIS CELL WILL RUN FOR APPROX. 5-8 MINUTES! PLEASE BE PATIENT.
For further documentation on SageMaker Clarify , you can refer the documentation [here](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-fairness-and-explainability.html)

In [48]:
import boto3
sm_client = boto3.client('sagemaker')

models = sm_client.list_models()
for model in models['Models']:
    print(model['ModelName'])

xgboost-2025-02-17-06-15-59-294
xgboost-2025-02-17-06-12-36-993
xgboost-2025-02-17-05-07-19-197
xgboost-2025-02-17-04-32-36-355
xgboost-2025-02-17-04-08-52-496
xgboost-2025-02-17-02-49-29-137
xgboost-2025-02-17-01-13-03-516
xgboost-2025-02-17-01-02-51-463
xgboost-2025-02-17-01-00-46-401


In [47]:
# cell 31
clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

INFO:sagemaker.clarify:Analysis Config: {'dataset_type': 'text/csv', 'headers': ['CNTSCHID', 'CNTSTUID', 'SISCO', 'ST347Q01JA', 'ST347Q02JA', 'ST349Q01JA_0', 'ST349Q01JA_1', 'ST349Q01JA_2', 'ST349Q01JA_3', 'ST349Q01JA_4', 'ST350Q01JA', 'ST356Q01JA', 'ST322Q01JA', 'ST322Q02JA', 'ST322Q03JA', 'ST322Q04JA', 'ST322Q06JA', 'ST322Q07JA', 'DURECEC', 'EFFORT1', 'EFFORT2', 'ST259Q01JA', 'WB164Q01HA', 'HOMEPOS', 'ST004D01T', 'GRADE', 'REPEAT', 'EXPECEDU', 'ICTAVSCH', 'ICTAVHOM', 'ICTDISTR', 'IMMIG', 'TARDYSD', 'ST226Q01JA', 'ST016Q01NA', 'MISSSC', 'Option_UH', 'OECD', 'PAREDINT', 'BMMJ1', 'BFMJ2', 'WB163Q06HA', 'WB163Q07HA', 'ST230Q01JA', 'SKIPPING', 'IC180Q01JA', 'IC180Q08JA', 'ST059Q02JA', 'ST296Q04JA', 'WB176Q01HA', 'STUDYHMW', 'IC184Q01JA', 'IC184Q02JA', 'IC184Q03JA', 'IC184Q04JA', 'ST059Q01TA', 'ST296Q01JA', 'ST272Q01JA', 'ST268Q01JA', 'ST268Q04JA', 'ST268Q07JA', 'ST293Q04JA', 'ST297Q01JA', 'ST297Q03JA', 'ST297Q05JA', 'ST297Q06JA', 'ST297Q07JA', 'ST297Q09JA', 'WB165Q01HA', 'WB166Q01HA', 'WB

.............................sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
We are not in a supported iso region, /bin/sh exiting gracefully with no changes.
INFO:sagemaker-clarify-processing:Starting SageMaker Clarify Processing job
INFO:analyzer.data_loading.data_loader_util:Analysis config path: /opt/ml/processing/input/config/analysis_config.json
INFO:analyzer.data_loading.data_loader_util:Analysis result path: /opt/ml/processing/output
INFO:analyzer.data_loading.data_loader_util:This host is algo-1.
INFO:analyzer.data_loading.data_loader_util:This host is the leader.
INFO:analyzer.data_loading.data_loader_util:Number of hosts in the cluster is 1.
INFO:sagemaker-clarify-processing:Running Python / Pandas based analyzer.
INFO:analyzer.data_loading.data_loader_factory:Dataset type: text/csv uri: /opt/ml/processing/input/data
INFO:sagema

KeyboardInterrupt: 

### Viewing the Explainability Report

Once the job is complete, you can view the explainability report in Studio under the 'Experiments and trials' tab

Look out for a trial component named 'clarify-explainability-' and see the Explainability tab. 

If you're not a Studio user yet, you can access this report at the following S3 bucket.

The report contains global explanations for the model with the input dataset

In [ ]:
# cell 32
explainability_output_path

In [ ]:
# cell 33
run_explainability_job_name = clarify_processor.latest_job.job_name
run_explainability_job_name


In [ ]:
# cell 34
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, run_explainability_job_name
        )
    )
)

In [ ]:
# cell 35
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, run_explainability_job_name
        )
    )
)

In [ ]:
# cell 36
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?prefix={}/">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, explainability_report_prefix
        )
    )
)

In [ ]:
# cell 37
explainability_output_path

### Download report from S3

In [ ]:
# cell 38
!aws s3 ls $explainability_output_path/


In [ ]:
# cell 39
!aws s3 cp --recursive $explainability_output_path ./explainability_report/

#### View the explainability pdf report below to see global explanations with SHAP for the model. The report also includes a SHAP summary plot for all individual  instances in the dataset.

In [ ]:
# cell 40
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="./explainability_report/report.html">Explainability Report</a></b>'))



### View the Explainability Report in SageMaker Studio

you can also view the explainability report in Studio under the experiments tab

![title](explainability_detail.gif)

### Analyze the local explanations of individual predictions by Clarify 

#### The pre-requisite for this section is that you have generated individual predictions for the test dataset by running inference with a SageMaker endpoint in the optional sections earlier
In this section, we will analyze and understand the local explainability results for each individual prediction produced by Clarify. Clarify produces a CSV file which contains the SHAP value for each feature per prediction. Let us download the CSV.

In [ ]:
# cell 41
from sagemaker.s3 import S3Downloader
import json
import io

# read the shap values
S3Downloader.download(s3_uri=explainability_output_path + "/explanations_shap", local_path="output")
shap_values_df = pd.read_csv("output/out.csv")
print(shap_values_df.shape)

Note that by default SHAP explains classifier models in terms of their margin output, before the logistic link function. That means the units of SHAP output and are log-odds units, so negative values imply probabilities of less than 0.5 meaning bad credit class (class 0). 

#### A brief technical summary of prediction output before the logistic link function and SHAP values

y = f(x) is the log-odd (logit) unit for the prediction output

E(y) is the log-odd (logit) unit for the prediction on the input baseline

SHAP values are in log-odd units as well 

The following is expected to hold true for every individual prediction : 

sum(SHAP values) + E(y)) == model_prediction_logit

logistic(model_prediction_logit) = model_prediction_probability

E(y) < 0 implies baseline probability less than 0.5 (bad credit baseline)

E(y) > 0 implies baseline probability greater than 0.5 (good credit baseline)

y < 0 implies predicted probability less than 0.5 (bad credit)

y > 0 implies predicted probability greater than 0.5 (good credit) 




We can retrieve E(y) , the log-odd unit of the prediction for the baseline input

In [ ]:
# cell 42
# get the base expected value to be used to plot SHAP values
S3Downloader.download(s3_uri=explainability_output_path + "/analysis.json", local_path="output")

with open("output/analysis.json") as json_file:
    data = json.load(json_file)
    base_value = data["explanations"]["kernel_shap"]["label0"]["expected_value"]

print("E(y): ", base_value)

As described in the earlier section, we have a baseline representing good credit prediction to be used with SHAP to contrast and explain bad credit predictions. E(y) > 0  implies baseline probability greater than 0.5 (good credit baseline). 

### Create a dataframe containing the model predictions generated earlier during inference

In [ ]:
# cell 43
from pandas import DataFrame

predictions_df = DataFrame(predictions, columns=["probability_score"])

predictions_df

### Join the predictions, SHAP value and test data

Now, we create a single dataframe containing all test data rows, with their corresponding SHAP values and prediction score.

In [ ]:
# cell 44
# join the probability score and shap values together in a single data frame
predictions_df.reset_index(drop=True, inplace=True)
shap_values_df.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

prediction_shap_df = pd.concat([predictions_df, shap_values_df, test_data], axis=1)
prediction_shap_df["probability_score"] = pd.to_numeric(
    prediction_shap_df["probability_score"], downcast="float"
)

prediction_shap_df

### Convert the probability score to binary prediction

Now, convert the probability scores to a binary value(1/0), based on a threshold(0.5), where probability scores greater than 0.5 are positive outcomes (good credit) and lesser are negative outcomes (bad credit).

In [ ]:
# cell 45
# create a new column as 'Prediction' converting the probability score to either 1 or 0
prediction_shap_df.insert(
    0, "Prediction", (prediction_shap_df["probability_score"] > 0.5).astype(int)
)

prediction_shap_df

### Filter for bad credit predictions only

Since we interested in explaining negative outcomes (bad credit predictions) only in this use case, we filter the records to keep only the record with prediction as 0.

In [ ]:
# cell 46
bad_credit_outcomes_df = prediction_shap_df[prediction_shap_df.iloc[:, 0] == 0]
bad_credit_outcomes_df

### Create SHAP plots 

Now we try to create some additional SHAP plots to understand how much different features contributed to a specific negative outcome.

#### Install open source SHAP library for more visualizations

In [ ]:
# cell 47
!conda update -n base -c defaults conda -y
!pip uninstall numpy -y
!pip uninstall numba -y
!pip install numba
!conda install -c conda-forge shap -y

In [ ]:
# cell 48
import shap

#### SHAP explanation plot for a single bad credit ensemble prediction instance. We will select the prediction instance with the lowest probability. 

In [ ]:
# cell 49
import matplotlib.pyplot as plt

min_index = prediction_shap_df["probability_score"].idxmin()
print(min_index)
print("mean probability of dataset")
print(prediction_shap_df[["probability_score"]].mean())
print("individual probability")
print(prediction_shap_df.iloc[min_index, 1])
print("sum of shap values")
print(prediction_shap_df.iloc[min_index, 2:22].sum())
print("base value from analysis.json")
print(base_value)

Example 'bad credit' prediction SHAP values.

In the chart below, f(x) is the prediction of this particular individual instance in log-odd units. If negative, it means it is a bad credit prediction. 

In the chart below, E(f(x)) is the prediction of the baseline input in log-odd units. It is positive , which means it belongs to the good credit class. 

The individual example is contrasted against the good credit baseline. So the features with negative SHAP values drive the final negative decision from the initial baseline positive value.




### In the below example, the input features (status = 1) , (purpose = 0) and (personal_status_sex = 2) are the top 3 features driving the negative decision. 

You can refer the data description to understand the mapping of these values to logical categories. 

In [ ]:
# cell 50
import inspect
import shap.plots._waterfall
source = inspect.getsource(shap.plots._waterfall)
new_source = source.replace("as pl", "as plt")
exec(new_source, shap.plots._waterfall.__dict__)

explanation_obj = shap._explanation.Explanation(
    values=prediction_shap_df.iloc[min_index, 2:22].to_numpy(),
    base_values=base_value,
    data=test_data.iloc[min_index].to_numpy(),
    feature_names=test_data.columns,
)
shap.plots.waterfall(shap_values=explanation_obj, max_display=20, show=False)

Feel free to change the min_index in the plot above to explain predictions of other individual instances

### Detect data bias with Amazon SageMaker Clarify
#### Amazon Science: [How Clarify helps machine learning developers detect unintended bias](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)

#### [Clarify Terms for Bias and Fairness](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-detect-data-bias.html) 

#### [Pre-training bias metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-data-bias.html)   

#### [Post-training bias metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html)

#### Calculate pre-training and post-training Bias metrics

Note: You can also execute pre-training and post-training bias detection jobs separately

A DataConfig object communicates some basic information about data I/O to Clarify. We specify where to find the input dataset, where to store the output, the target column (label), the header names, and the dataset type.

Similarly, the ModelConfig (created earlier for the explainability job) object communicates information about your trained model and ModelPredictedLabelConfig provides information on the format of your predictions.

In [ ]:
# cell 51
bias_report_prefix = "{}/clarify-bias".format( prefix)
bias_report_output_path = "s3://{}/{}".format(bucket,bias_report_prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_raw,
    s3_output_path=bias_report_output_path,
    label="credit_risk",
    headers=training_data.columns.to_list(),
    dataset_type="text/csv",
)
predictions_config = clarify.ModelPredictedLabelConfig(label=None, probability=0)

SageMaker Clarify also needs the sensitive columns (facets) and the desirable outcomes (facet_values_or_threshold).

We specify this information in the BiasConfig API. Here  age is the facet that we analyze and 40 is the threshold. The group 'personal_status_sex' is used to form subgroups for the measurement of Conditional Demographic Disparity (CDD) metric only.

In [ ]:
# cell 52
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1],
    facet_name="age",
    facet_values_or_threshold=[40],
    group_name="personal_status_sex",
)

#### NOTE: THIS CELL WILL RUN FOR APPROX. 5-8 MINUTES! PLEASE BE PATIENT.
For further documentation on SageMaker Clarify, you can refer the documentation [here](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-fairness-and-explainability.html)

In [ ]:
# cell 53
clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

#### Viewing the Bias detection Report
You can view the bis detection report in Studio under the experiments tab 

If you're not a Studio user yet, you can access this report at the following S3 bucket.

In [ ]:
# cell 54
bias_report_output_path

In [ ]:
# cell 55
run_post_training_bias_processing_job_name = clarify_processor.latest_job.job_name
run_post_training_bias_processing_job_name

In [ ]:
# cell 56

from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, run_post_training_bias_processing_job_name
        )
    )
)



In [ ]:
# cell 57
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, run_post_training_bias_processing_job_name
        )
    )
)

In [ ]:
# cell 58
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?prefix={}/">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, bias_report_prefix
        )
    )
)

#### Download Report From S3

In [ ]:
# cell 59
!aws s3 ls $bias_report_output_path/

In [ ]:
# cell 60
bias_report_output_path

In [ ]:
# cell 61
!aws s3 cp --recursive $bias_report_output_path ./generated_bias_report/

#### View the bias report pdf that contains the pre-training bias and post-training bias metrics. 

In [ ]:
# cell 62
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="./generated_bias_report/report.html">Bias Report</a></b>'))

### View Bias Report in Studio

Alternatively, you can also view the bias report in Studio under the experiments tab. Each bias metric has detailed explanations with examples that you can explore. You could also summarize the results in a handy table!

![title](bias_report.gif)

Let us specifically look at a couple of pre-training and post-training bias metrics. 

Pre-training bias metrics
1. Class imbalance
2. DPL - Difference in positive proportions in true labels 

Post-training bias metrics
1. DPPL - Difference in positive proportions in predicted labels
2. DI - Disparate Impact

In [ ]:
# cell 63
S3Downloader.download(s3_uri=bias_report_output_path + "/analysis.json", local_path="output")

with open("output/analysis.json") as json_file:
    data = json.load(json_file)
    print("pre-training bias metrics")
    class_imbalance = data["pre_training_bias_metrics"]["facets"]["age"][0]["metrics"][1]["value"]
    print("class imbalance: ", class_imbalance)
    DPL = data["pre_training_bias_metrics"]["facets"]["age"][0]["metrics"][2]["value"]
    print("DPL: ", DPL)
    print("\n")
    print("post training bias metrics")
    DPPL = data["post_training_bias_metrics"]["facets"]["age"][0]["metrics"][6]["value"]
    print("DPPL: ", DPPL)
    DI = data["post_training_bias_metrics"]["facets"]["age"][0]["metrics"][5]["value"]
    print("DI: ", DI)

Here, we see that for the "age" facet with threshold of [40] , [CI](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-bias-metric-class-imbalance.html) and [DI](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-post-training-bias-metric-di.html) are high , whereas [DPL](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-data-bias-metric-true-label-imbalance.html) and [DPPL](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-post-training-bias-metric-dppl.html) are low. Data pre-processing techniques can be applied to mitigate the pre-training bias and training algorithms can be re-evaluated to mitigate the post-training bias. 

### 6. Clean Up
Finally, don't forget to clean up the resources we set up and used for this demo!

In [ ]:
# cell 64
session.delete_endpoint(endpoint_name)

In [ ]:
# cell 65
session.delete_model(pipeline_model.name)

## 7. Additional Resources to explore 

* [Working toward fairer machine learning](https://www.amazon.science/research-awards/success-stories/algorithmic-bias-and-fairness-in-machine-learning)
* [Fairness Measures for Machine Learning in Finance](https://pages.awscloud.com/rs/112-TZM-766/images/Fairness.Measures.for.Machine.Learning.in.Finance.pdf)
* [Amazon SageMaker Clarify: Machine learning bias detection and explainability in the cloud](https://www.amazon.science/publications/amazon-sagemaker-clarify-machine-learning-bias-detection-and-explainability-in-the-cloud)
* [Amazon AI Fairness and Explainability Whitepaper](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)
* [How Clarify helps machine learning developers detect unintended bias](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)